In [2]:
import json
from utils.gho_filter import filter_gho_users

with open('results.json', 'r') as fp:
    res = json.load(fp)
data = []
for entry in res:
    data += entry['data']['users']

gho_users = set(filter_gho_users(data))

In [4]:
def get_collats(history):
    return list(set([entry['reserve']['symbol'] for entry in history]))

collaterals = []
for entry in data:
    if entry['id'] in gho_users:
        collaterals += get_collats(entry['supplyHistory'])

# GHO users

In [13]:
import pandas as pd
pd.Series(collaterals).value_counts() / sum(pd.Series(collaterals).value_counts()) * 100

WETH      25.924590
wstETH    13.187804
WBTC      12.375970
USDC      12.285766
AAVE       6.675086
USDT       6.061699
LINK       5.899332
DAI        4.600397
rETH       3.572073
sDAI       1.695833
cbETH      1.280895
CRV        0.865957
LUSD       0.757712
UNI        0.667509
MKR        0.631427
SNX        0.577305
crvUSD     0.414938
           0.360815
RPL        0.342775
LDO        0.324734
BAL        0.288652
ENS        0.234530
USDe       0.180408
FRAX       0.180408
ETHx       0.126285
1INCH      0.108245
PYUSD      0.108245
osETH      0.072163
weETH      0.072163
sUSDe      0.054122
KNC        0.054122
FXS        0.018041
dtype: float64

# Liquidated GHO users

In [16]:
def is_gho_liq(history):
    return any([entry for entry in history if entry['principalReserve']['symbol'] == "GHO"])

gho_liqs = [d for d in data if is_gho_liq(d['liquidationCallHistory'])]
len(gho_liqs)

152

In [20]:
def get_collats(history):
    return list(set([entry['reserve']['symbol'] for entry in history]))

gho_liq_collaterals = []
for entry in gho_liqs:
    gho_liq_collaterals += get_collats(entry['supplyHistory'])

In [21]:
pd.Series(gho_liq_collaterals).value_counts() / sum(pd.Series(gho_liq_collaterals).value_counts()) * 100

WETH      26.730310
wstETH    13.126492
WBTC      10.739857
USDC       9.069212
LINK       8.830549
AAVE       8.591885
USDT       5.727924
DAI        4.057279
rETH       2.863962
cbETH      2.147971
CRV        1.193317
MKR        1.193317
sDAI       0.954654
SNX        0.715990
LUSD       0.715990
BAL        0.715990
ETHx       0.477327
RPL        0.477327
           0.477327
osETH      0.238663
LDO        0.238663
FRAX       0.238663
crvUSD     0.238663
USDe       0.238663
dtype: float64

# V3 Users (non GHO)

In [8]:
non_gho_collaterals = []
for entry in data:
    if entry['id'] not in gho_users:
        non_gho_collaterals += get_collats(entry['supplyHistory'])

In [12]:
pd.Series(non_gho_collaterals).value_counts() / sum(pd.Series(non_gho_collaterals).value_counts()) * 100

WETH      38.714968
USDC      15.069983
USDT      10.862858
WBTC       9.882116
wstETH     6.509596
DAI        4.996265
LINK       3.695645
AAVE       2.279739
rETH       1.430520
CRV        0.821615
cbETH      0.626766
UNI        0.545579
MKR        0.539084
LDO        0.474134
sDAI       0.464391
SNX        0.384828
LUSD       0.378333
crvUSD     0.319878
BAL        0.297145
           0.282532
USDe       0.276037
RPL        0.246809
1INCH      0.151008
ENS        0.141266
FRAX       0.123405
weETH      0.115286
PYUSD      0.110415
sUSDe      0.089306
FXS        0.045465
KNC        0.038970
ETHx       0.037346
osETH      0.025980
STG        0.022732
dtype: float64

# V3 Borrowers (non GHO)

In [9]:
non_gho_bor_collaterals = []
for entry in data:
    if entry['id'] not in gho_users and entry['borrowHistory']:
        non_gho_bor_collaterals += get_collats(entry['supplyHistory'])

In [14]:
(pd.Series(non_gho_bor_collaterals).value_counts() / sum(pd.Series(non_gho_bor_collaterals).value_counts()) * 100)[:8].to_dict()

{'WETH': 37.129972247918595,
 'WBTC': 15.364824236817762,
 'wstETH': 10.343432007400555,
 'USDC': 9.664084181313598,
 'USDT': 5.764338575393155,
 'LINK': 5.657377428307123,
 'DAI': 4.3825161887141535,
 'AAVE': 2.8301341350601295}

In [15]:
sum(_.values())

91.13667900092507

In [19]:
res = {}
res['All v3 Borrowers (excl. GHO)'] = {'WETH': 37.129972247918595,
 'WBTC': 15.364824236817762,
 'wstETH': 10.343432007400555,
 'USDC': 9.664084181313598,
 'USDT': 5.764338575393155,
 'LINK': 5.657377428307123,
 'DAI': 4.3825161887141535,
 'AAVE': 2.8301341350601295,
  'Others': 100 - 91.13667900092507}

import pandas as pd
tokens_of_interest = ["DAI", "crvUSD", "USDC", "USDT", "WETH", "WBTC", "GHO", "LUSD", "LINK", "CRV",]
collats_of_interest = ["WETH", "WBTC", "wstETH", "USDC", "USDT", "LINK", "DAI", "AAVE"]
for token in tokens_of_interest:
    token_bor_collaterals = []
    for entry in data:
        if entry['borrowHistory']:
            if any([h['reserve']['symbol'] == token for h in entry['borrowHistory']]):
                token_bor_collaterals += get_collats(entry['supplyHistory'])
    print(token)
    print((pd.Series(token_bor_collaterals).value_counts() / sum(pd.Series(token_bor_collaterals).value_counts()) * 100)[:8].to_dict())
    print("*"*40)
    top_8 = (pd.Series(token_bor_collaterals).value_counts() / sum(pd.Series(token_bor_collaterals).value_counts()) * 100)[:8].to_dict()
    res[token + ' Borrowers'] = {k: v for k, v in top_8.items() if k in collats_of_interest}
    res[token + ' Borrowers']['Others'] = 100 - sum(res[token + ' Borrowers'].values())
res

DAI
{'WETH': 28.99237933954276, 'WBTC': 15.867908552074514, 'wstETH': 10.719729043183742, 'USDC': 8.399661303979679, 'DAI': 7.3835732430143945, 'LINK': 6.452159187129551, 'USDT': 5.707027942421677, 'AAVE': 3.6917866215071973}
****************************************
crvUSD
{'WETH': 20.273348519362187, 'WBTC': 17.084282460136674, 'wstETH': 12.072892938496583, 'USDC': 10.478359908883828, 'USDT': 6.605922551252847, 'AAVE': 6.605922551252847, 'rETH': 4.783599088838269, 'LINK': 4.100227790432802}
****************************************
USDC
{'WETH': 35.33340182882975, 'WBTC': 16.783109010582553, 'wstETH': 11.060310284598787, 'USDC': 9.878762971334634, 'LINK': 6.621802116510839, 'USDT': 4.114866947498202, 'DAI': 3.570327750950375, 'AAVE': 3.544641939792459}
****************************************
USDT
{'WETH': 35.8081545546445, 'WBTC': 18.594855666629226, 'wstETH': 9.698977872627204, 'USDT': 7.143659440637987, 'USDC': 6.879703470740201, 'LINK': 6.778613950353813, 'AAVE': 4.066045153319106,

{'All v3 Borrowers (excl. GHO)': {'WETH': 37.129972247918595,
  'WBTC': 15.364824236817762,
  'wstETH': 10.343432007400555,
  'USDC': 9.664084181313598,
  'USDT': 5.764338575393155,
  'LINK': 5.657377428307123,
  'DAI': 4.3825161887141535,
  'AAVE': 2.8301341350601295,
  'Others': 8.86332099907493},
 'DAI Borrowers': {'WETH': 28.99237933954276,
  'WBTC': 15.867908552074514,
  'wstETH': 10.719729043183742,
  'USDC': 8.399661303979679,
  'DAI': 7.3835732430143945,
  'LINK': 6.452159187129551,
  'USDT': 5.707027942421677,
  'AAVE': 3.6917866215071973,
  'Others': 12.785774767146492},
 'crvUSD Borrowers': {'WETH': 20.273348519362187,
  'WBTC': 17.084282460136674,
  'wstETH': 12.072892938496583,
  'USDC': 10.478359908883828,
  'USDT': 6.605922551252847,
  'AAVE': 6.605922551252847,
  'LINK': 4.100227790432802,
  'Others': 22.779043280182236},
 'USDC Borrowers': {'WETH': 35.33340182882975,
  'WBTC': 16.783109010582553,
  'wstETH': 11.060310284598787,
  'USDC': 9.878762971334634,
  'LINK': 6.